In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('mode.chained_assignment', None)
warnings.simplefilter("ignore", category=RuntimeWarning)

import random
import os
from juno_classes import *

from scipy.integrate import trapz

import matplotlib.colors as clrs
from matplotlib.cm import ScalarMappable



def extract_base_exponent_scientific(number):
    # Calculate exponent with respect to base 10
    exponent = int(np.log10(number))
    base = number / (10 ** exponent)
    return base, exponent

In [ ]:
def mean_field_align(data_df, window_size=10):
    """Rotate magnetometer data into a mean-field-aligned coordinate system.
        Using the methods described by Khurana & Kivelson[1989]

    Parameters
    ----------
    window_size : int, optional
        The size of the window in minutes that is moved over data to average over.
            This should be EVEN to ensure times of MFA and regular data line up.
            The default is 24.

    Returns
    -------
    None.

    """
    # A windows of size 'window_size' in minutes is then moved along the data
    # An average inside of the window is found for each entry
    #print(self.data_df)
    mean_mag_data = pd.DataFrame({'MEAN_BX': [], 'MEAN_BY': [], 'MEAN_BZ': []})
    finish_datetime = (data_df.index[-1]
    - timedelta(minutes=np.floor(window_size/2)))
    for datetime_index in data_df.index:
        start_datetime = datetime_index
        end_datetime = start_datetime + timedelta(minutes=window_size)
        mean_datetime_index = pd.DatetimeIndex([
            (start_datetime + timedelta(minutes=round(window_size/2))).isoformat()
        ])
        temp_mean = data_df[start_datetime.isoformat(): end_datetime.isoformat()].mean()
        mean_array_to_concat = pd.DataFrame(
            {'MEAN_BX': temp_mean.BX_JSS,
                'MEAN_BY': temp_mean.BY_JSS,
                'MEAN_BZ': temp_mean.BZ_JSS}, index=mean_datetime_index)
        mean_mag_data = pd.concat([mean_mag_data, mean_array_to_concat])
        
        if mean_datetime_index == finish_datetime:
            break
    # mean_mag_data and data_df are cut to align the time series of each
    # mag_data loses half of the time_window in the front
    # mean_mag_data loses half of the time window in the end
    # The two dataframes are then concatenated into one for simplicity
    #data_df = data_df.drop(data_df[: (data_df.index[0] +
    #     timedelta(minutes=round(window_size / 2) - 1)
    # ).isoformat()].index)
    




    data_df = data_df[mean_mag_data.index[0].isoformat(): 
                                mean_mag_data.index[-1].isoformat()]
    data_df = pd.concat([data_df, mean_mag_data], axis=1)
    del mean_mag_data

    # The perturbation components of the mean field are found.

    return data_df

def mean_changer(df):
    mean_x = df['MEAN_BX']
    mean_y = df['MEAN_BY']
    mean_z = df['MEAN_BZ']
    raw_x = df['BX_JSS']
    raw_y = df['BY_JSS']
    raw_z = df['BZ_JSS']
        
    mean_vecs = df[['MEAN_BX', 'MEAN_BY', 'MEAN_BZ']]
    mean_magnitude = np.sqrt((mean_vecs**2).sum(axis=1))

    
    perturbed_x = raw_x - mean_x
    perturbed_y = raw_y - mean_y
    perturbed_z = raw_z - mean_z
    
    perturbed_df = pd.DataFrame({'B_x_pert': perturbed_x, 'B_y_pert': perturbed_y, 'B_z_pert': perturbed_z})
    perturbed_df_perp = pd.DataFrame({'B_x_pert_perp': [], 'B_y_pert_perp': [], 'B_z_pert_perp': []})
    perturbed_df_par = pd.DataFrame({'B_x_pert_par': [], 'B_y_pert_par': [], 'B_z_pert_par': []})
    par_mag_df = pd.DataFrame({'B_pert_par': []})
                                    
    for i in range(len(perturbed_x)):
        new_df_index = pd.DatetimeIndex([df.index[i].isoformat()])
        mean_vecs = df[['MEAN_BX', 'MEAN_BY', 'MEAN_BZ']]
        parallel_hat = mean_vecs.iloc[i] / mean_magnitude[i]
        perturbed_parallel_mag = np.dot((perturbed_df.iloc[i].to_numpy()), parallel_hat)
        perturbed_parallel_vector = np.asarray(perturbed_parallel_mag*parallel_hat)
        perturbed_par_x = perturbed_parallel_vector[0]
        perturbed_par_y = perturbed_parallel_vector[1]
        perturbed_par_z = perturbed_parallel_vector[2]
        perturbed_perp_x = perturbed_df.iloc[i]['B_x_pert'] - perturbed_par_x
        perturbed_perp_y = perturbed_df.iloc[i]['B_y_pert'] - perturbed_par_y
        perturbed_perp_z = perturbed_df.iloc[i]['B_z_pert'] - perturbed_par_z
        
        
        perturbed_df_perp = pd.concat([perturbed_df_perp,pd.DataFrame({'B_x_pert_perp': perturbed_perp_x, 'B_y_pert_perp': perturbed_perp_y, 'B_z_pert_perp': perturbed_perp_z},
                             index=new_df_index)])
    
        perturbed_df_par = pd.concat([perturbed_df_par, pd.DataFrame({'B_x_pert_par': perturbed_par_x, 'B_y_pert_par': perturbed_par_y, 'B_z_pert_par': perturbed_par_z},
                        index=new_df_index) ])
        
        par_mag_df = pd.concat([par_mag_df,         pd.DataFrame({'B_pert_par': perturbed_parallel_mag},
                        index=new_df_index)])
        
    data_df = pd.concat([df, perturbed_df_perp], axis=1)
    data_df  = pd.concat([data_df, perturbed_df_par], axis=1)
    data_df = pd.concat([data_df, par_mag_df], axis = 1)
    return data_df
        
        
        

In [ ]:
startOrbit = 50
endOrbit = 54

for i in range(startOrbit, endOrbit+1):
    print('grabbing orbit ' + str(i))
    filename = '/home/mtfranciscovich/Juno-codes/reduced_inbound_1s_pickles_unit_z_mag/reduced_orbit_' +str(i)+'_1s.pkl'
    picklefile = open(filename,'rb')
    rawdata = pickle.load(picklefile)
    
    print('grabbing mean v orbit ' + str(i))
    #filename = '/home/mtfranciscovich/Juno-codes/test_fits_u_cart_means_4min/u_cart_means_4min_orbit_' +str(i) +'.pkl'
    filename = '/home/mtfranciscovich/Juno-codes/fits_u_cart_pickles_updated/u_cart_orbit_' +str(i)+'.pkl'
    picklefile = open(filename,'rb')
    mean_v_data = pickle.load(picklefile)
    
    
    downsampled = pd.concat([rawdata, mean_v_data.dropna()], axis=1) 
    
    
    
    downsampled = downsampled.dropna()
    
    print('mean field...')
    meandata = mean_field_align(downsampled)
    print('running mean changer...')
    final_df = mean_changer(meandata)
    final_df.to_pickle('/home/mtfranciscovich/Juno-codes/means_10min_reduced_inbound_1s_pickles_unit_z_mag_jss_downsampled/reduced_orbit_'+str(i)+'_1s.pkl')

    